<a href="https://colab.research.google.com/github/wangjb/cwb_open_data/blob/master/rt_meteo_info_manned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def retrieve_data(datacode,auth):
  import urllib
  # query string to CWB open data service
  url='https://opendata.cwb.gov.tw/'
  sub_path='fileapi/v1/opendataapi/'
  # instant 10 min precip
  datalimit=''
  dataoffset=''
  dataformat='JSON'
  stationID=''
  datastatus=''

  # create query string
  url_path=url+sub_path+datacode
  query_fields=['Authorization','limit','offset','format','stationID','status']
  query_data=(auth,datalimit,dataoffset,dataformat,stationID,datastatus)
  query_string=dict(zip(query_fields,query_data))

  # request data
  import requests
  r = requests.get(url_path,params=query_string,stream=True)
  jsondata = r.json()

  return jsondata

In [2]:
datacode = 'O-A0003-001'
auth = None
jsondata = retrieve_data(datacode, auth)

In [3]:
jsondata['cwbopendata']['location'][0].keys()

dict_keys(['lat', 'lon', 'lat_wgs84', 'lon_wgs84', 'locationName', 'stationId', 'time', 'weatherElement', 'parameter'])

In [4]:
import pandas as pd
data = pd.DataFrame(jsondata['cwbopendata']['location'])

# process obstime
data['obstime'] = pd.DataFrame(data['time'].to_list())

# process weatherElement
weatherElement = pd.DataFrame(data['weatherElement'].to_list())
weatherElement.columns = [ x['elementName'] for x in weatherElement.iloc[0] ]
weatherElement = weatherElement.applymap(lambda x: x['elementValue']['value'])

# process parameter
parameter = pd.DataFrame(data['parameter'].to_list())
parameter.columns = [ x['parameterName'] for x in parameter.iloc[0] ]
parameter = parameter.applymap(lambda x: x['parameterValue'])

# merge data
data = data.merge(weatherElement,left_index=True,right_index=True)
data = data.merge(parameter,left_index=True,right_index=True)
data = data.drop(['time','weatherElement','parameter'],axis=1)

In [5]:
data

,lat,lon,lat_wgs84,lon_wgs84,locationName,stationId,obstime,ELEV,WDIR,WDSD,TEMP,HUMD,PRES,24R,H_FX,H_XD,H_FXT,H_F10,H_10D,H_F10T,H_UVI,D_TX,D_TXT,D_TN,D_TNT,D_TS,VIS,Weather,CITY,CITY_SN,TOWN,TOWN_SN
0,25.135130,121.732265,25.13331389,121.740475,基隆,466940,2020-08-12T16:50:00+08:00,26.70,300,3.30,32.20,0.56,1009.30,0,6.40,210,1559,3.50,180,1600,1.88,35.70,1257,26.80,0535,9.40,無觀測,-99,基隆市,03,仁愛區,015
1,25.166689,121.440690,25.16488889,121.44890560,淡水,466900,2020-08-12T16:50:00+08:00,19,20,0.90,29.60,0.77,1011.30,0.50,6.10,10,1507,3,360,1515,-99,32.70,1453,25.30,0246,8.90,無觀測,-99,新北市,06,淡水區,004
2,24.999447,121.433812,24.99764722,121.44201670,板橋,466880,2020-08-12T16:50:00+08:00,11,200,1.30,28,0.86,1011.90,1,4.70,210,1546,2.50,220,1555,-99,35.40,1237,24.90,0518,6.90,無觀測,-99,新北市,06,板橋區,041
3,25.163883,121.536332,25.16207778,121.54454720,竹子湖,466930,2020-08-12T16:50:00+08:00,607.10,260,0.80,-99,-99,946.20,17,5.60,70,1514,2.60,50,1522,-99,-99,-99,-99,-99,7,無觀測,-99,臺北市,01,北投區,006
4,25.184391,121.521515,25.18258611,121.52973060,鞍部,466910,2020-08-12T16:50:00+08:00,832.60,350,2.90,23.30,0.96,-99,10,4.80,360,1551,3.20,360,1554,1.52,28.80,1155,20.30,0533,7.10,無觀測,-99,臺北市,01,北投區,006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,24.2739699999999980,120.593079,24.2739699999999980,120.593079,國三S173K,CAF070,2020-08-12T16:50:00+08:00,181,90,0.60,27.20,0.77,-99,2.50,8,280,1501,6.50,280,1501,-99,30.80,1103,25.10,0533,-99,-99,-99,臺中市,02,清水區,112
70,24.2267210000000010,120.58905799999999,24.2267210000000010,120.58905799999999,國三S178K,CAF080,2020-08-12T16:50:00+08:00,208,-99,-99,26.20,0.92,-99,12.50,-99,-99,-99,-99,-99,-99,-99,32,1112,25.20,0531,-99,-99,-99,臺中市,02,沙鹿區,120
71,24.092468,120.603256,24.092468,120.603256,國三S202K,CAG060,2020-08-12T16:50:00+08:00,48,90,4.30,25,0.96,-99,8.50,9.50,260,1507,5.40,250,1510,-99,33,1201,25,0510,-99,-99,-99,彰化縣,14,彰化市,142
72,24.0068300000000010,120.65321400000001,24.0068300000000010,120.65321400000001,國三S217K,CAH020,2020-08-12T16:50:00+08:00,93,-99,-99,25.10,0.89,-99,5.50,-99,-99,-99,-99,-99,-99,-99,33.60,1110,24.30,0337,-99,-99,-99,南投縣,13,草屯鎮,152
